- Load a dataset

- Make it as a iterator

- Load a tokeniser

- Write a processing function 

- Map it to the dataset 

- Create a new tokeniser

- Train it with the dataset

- Write the post processing function

- Run the evaluation 

- work on multiple peft configuration

- load the model for inference

- load the model for training

- start the training with t5-small

In [1]:
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments
)
from datasets import load_dataset
from evaluate import load

In [3]:
data_path = "jackhhao/jailbreak-classification"
model_path = "distilbert-base-cased"

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_path)

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [5]:
jb = load_dataset("jackhhao/jailbreak-classification")
jb

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['prompt', 'type'],
        num_rows: 1044
    })
    test: Dataset({
        features: ['prompt', 'type'],
        num_rows: 262
    })
})

In [6]:
jb['train'][0]

{'prompt': 'You are a devoted fan of a celebrity.', 'type': 'benign'}

In [18]:
jb['train'].features.type.num_fields

2

In [20]:
set([row['type'] for row in jb['train']])

{'benign', 'jailbreak'}

In [29]:
id2label = {0: "jailbreak", 1: "benign"}
label2id = {"jailbreak": 0, "benign": 1}

In [30]:
model_wt = AutoModelForSequenceClassification.from_pretrained(model_path,
                                                             num_labels=2,
                                                             id2label=id2label,
                                                             label2id=label2id)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [46]:
def pre_process(rows):
    tokens = tokenizer(rows['prompt'],
                       truncation=True,
                      padding=True)
    tokens['label'] = [label2id[text] for text in rows['type']]
    return tokens

In [47]:
processed_jb = jb.map(pre_process, batched=True)

Map:   0%|          | 0/1044 [00:00<?, ? examples/s]

Map:   0%|          | 0/262 [00:00<?, ? examples/s]

In [48]:
processed_jb

DatasetDict({
    train: Dataset({
        features: ['prompt', 'type', 'input_ids', 'attention_mask', 'label'],
        num_rows: 1044
    })
    test: Dataset({
        features: ['prompt', 'type', 'input_ids', 'attention_mask', 'label'],
        num_rows: 262
    })
})

In [49]:
len(processed_jb['train'][16]['input_ids'])

512

In [56]:
processed_jb['train'][10]['type']

'benign'

In [22]:
from transformers import (
    DefaultDataCollator,
    DataCollatorWithPadding
)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [59]:
targs = TrainingArguments(
    output_dir="/home/kamal/training_files/prac",
    evaluation_strategy='epoch',
    save_strategy='epoch',
    num_train_epochs=3,
    weight_decay=0.01,
    learning_rate=2e-5,
    push_to_hub=False,
    report_to="none",
    per_device_eval_batch_size=24,
    per_device_train_batch_size=24
)

In [24]:
import numpy as np
accuracy = load('accuracy')

def compute_acc(eval_pred):
    preds, refs = eval_pred
    preds = np.argmax(preds, axis=1)
    return accuracy.compute(predictions=preds,
                    references=refs)
    

In [60]:
trainer = Trainer(
    model=model_wt,
    args=targs,
    train_dataset=processed_jb['train'],
    eval_dataset=processed_jb['test'],
    compute_metrics=compute_acc,
    data_collator=data_collator,
    tokenizer=tokenizer
)

In [61]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.056307,0.977099
2,No log,0.059603,0.980916
3,No log,0.049662,0.980916


TrainOutput(global_step=132, training_loss=0.027702107574000503, metrics={'train_runtime': 50.7779, 'train_samples_per_second': 61.68, 'train_steps_per_second': 2.6, 'total_flos': 414887892590592.0, 'train_loss': 0.027702107574000503, 'epoch': 3.0})